In [ ]:
#setup for processed data
#Note: use for BinaryArray data produced from Entrainment_Preprocessing.ipynb
def GetProcessedString(PROCESSING=False):
    if PROCESSING==True:
        Processed_string="PROCESSED_"
    else:
        Processed_string=""
    return Processed_string

PROCESSING=False 
PROCESSING=True #set to True if using Turbulence-Removed Binary Arrays
Processed_string = GetProcessedString(PROCESSING=PROCESSING)

In [ ]:
####################################
#ENVIRONMENT SETUP

In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py

In [ ]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [ ]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [ ]:
#IMPORT FUNCTIONS
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
import FUNCTIONS_Variable_Calculation
from FUNCTIONS_Variable_Calculation import * # import NumericalFunctions 

In [ ]:
####################################
#LOADING CLASSES

In [ ]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=2)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="LagrangianArrays", dataName=f"{Processed_string}Lagrangian_Entrainment",
                                dtype='float32')

In [ ]:
#JOB ARRAY SETUP
UsingJobArray=True

def GetNumJobs(res,t_res):
    if res=='1km':
        if t_res=='5min':
            num_jobs=20
        elif t_res=='1min':
            num_jobs=100
    elif res=='250m': 
        if t_res=='1min':
            num_jobs=500
    return num_jobs
num_jobs = GetNumJobs(ModelData.res,ModelData.t_res)
SlurmJobArray = SlurmJobArray_Class(total_elements=ModelData.Ntime, num_jobs=num_jobs, UsingJobArray=UsingJobArray)
start_job = SlurmJobArray.start_job; end_job = SlurmJobArray.end_job

def GetNumElements():
    loop_elements = np.arange(ModelData.Ntime)[start_job:end_job]
    return loop_elements
loop_elements = GetNumElements()

In [ ]:
####################################
#FUNCTIONS

In [ ]:
def GetZYX(inputParcelDirectory, timeString):
    #Lagrangian Position Arrays
    ##############
    def grid_location(z,y,x):
        zf=ModelData.zf*1000; which_zh=np.clip(np.searchsorted(zf,z)-1,0,None).astype(np.uint16)
        #which_zh=np.where(which_zh == -1, 0, which_zh) 
        
        yf=ModelData.yf*1000; which_yh=np.clip(np.searchsorted(yf,y)-1,0,None).astype(np.uint16) 
        #which_yh=np.where(which_yh == -1, 0, which_yh) 
        
        xf=ModelData.xf*1000; which_xh=np.clip(np.searchsorted(xf,x)-1,0,None).astype(np.uint16)
        #which_xh=np.where(which_xh == -1, 0, which_xh) 
        
        return which_zh,which_yh,which_xh
    
    # print('Creating Lagrangian X,Y,Z Binary Arrays')
    x = DataManager.GetTimestepParcel(DataManager.inputParcelDirectory, timeString, variableName="x")
    y = DataManager.GetTimestepParcel(DataManager.inputParcelDirectory, timeString, variableName="y")
    z = DataManager.GetTimestepParcel(DataManager.inputParcelDirectory, timeString, variableName="z")
    Z,Y,X=grid_location(z,y,x)
    return z,x,Z,Y,X

In [ ]:
def MakeLagrangianArray(inputDataDirectory, timeString,varnames,Z,Y,X): #FAST ADVANCED INDEXING VERSION
    # print('Making Variables Lagrangian Array')
    # Initialize dictionaries
    var_data_dict = {varname: CallVariable(ModelData, DataManager, timeString, varname) for varname in varnames}
    VAR = {varname: np.zeros_like(Z, dtype='float32') for varname in varnames}

    # print('Running')
    # Nt = len(data['time'])
    # Get Indices
    z_idx = Z
    y_idx = Y
    x_idx = X
    # ts = np.arange(Nt)[:, None]
    # t_idx = np.broadcast_to(ts, Z.shape)

    for varname, var_data in var_data_dict.items():
        # VAR[varname] = var_data[t_idx, z_idx, y_idx, x_idx]
        VAR[varname] = var_data[z_idx, y_idx, x_idx]

    return [VAR[varname] for varname in varnames]
    # return VAR

In [ ]:
def Getvarnames():
    varnames=[f'{Processed_string}Entrainment_g',
              f'{Processed_string}Entrainment_c',
              f'{Processed_string}TransferEntrainment_g',
              f'{Processed_string}TransferEntrainment_c']
    varnames+=[f'{Processed_string}Detrainment_g',
               f'{Processed_string}Detrainment_c',
               f'{Processed_string}TransferDetrainment_g',
               f'{Processed_string}TransferDetrainment_c']
    return varnames

In [ ]:
def GetVariables(inputDataDirectory,timeString, Z,Y,X, Processed_string):
    # print('Making W and QCQI Lagrangian Array')
    varnames = Getvarnames()
    
    [E_g,E_c,
     TransferE_g,TransferE_c,
     D_g,D_c,
     TransferD_g,TransferD_c] = MakeLagrangianArray(inputDataDirectory,timeString, varnames, Z,Y,X)
    
    outputDictionary = {f"{Processed_string}E_g": E_g, 
                        f"{Processed_string}E_c": E_c,
                        f"{Processed_string}TransferE_g": TransferE_g,
                        f"{Processed_string}TransferE_c": TransferE_c,
                        f"{Processed_string}D_g": D_g, f"{Processed_string}D_c": D_c,
                        f"{Processed_string}TransferD_g": TransferD_g, 
                        f"{Processed_string}TransferD_c": TransferD_c,
                       }
    return outputDictionary

In [ ]:
###############################################################
#RUNNING

In [ ]:
#CALCULATING AND APPENDING TO DATA EACH TIMESTEP
for t in loop_elements:
    if np.mod(t,1)==0: print(f'Current time {t}')

    if t!=ModelData.Ntime-1: #detrainment doesn't exist at last timestep
        #getting timestring for loading input data
        timeString = ModelData.timeStrings[t]
    
        #loading input variables
        z,x,Z,Y,X = GetZYX(DataManager.inputParcelDirectory, timeString)
    
        #calculating variables
        outputDictionary = GetVariables(DataManager,timeString, Z,Y,X, Processed_string)
        
        #outputting
        DataManager.SaveOutputTimestep(DataManager.outputDataDirectory, timeString, outputDictionary)

In [ ]:
######################################################

In [ ]:
# #READING BACK IN
# def GetVariableData(t, dataType=f"{Processed_string}Lagrangian_Entrainment", dataName=f"{Processed_string}Lagrangian_Entrainment"):
#     res = ModelData.res
#     t_res = ModelData.t_res
#     Nz_str = ModelData.Nz_str
#     inputDirectory = os.path.join(DataManager.outputDirectory, f"{res}_{t_res}_{Nz_str}nz", dataType)
#     timeString = ModelData.timeStrings[t]

#     FileName = os.path.join(inputDirectory, f"{dataName}_{res}_{t_res}_{Nz_str}nz_{timeString}.h5")

#     dataDictionary = {}
#     with h5py.File(FileName, 'r') as f:
#         print("Keys in file:", list(f.keys()))
#         for key in f.keys():
#             dataDictionary[key] = f[key][:]
#             print(f"{key}: shape = {dataDictionary[key].shape}, dtype = {dataDictionary[key].dtype}")
#     return dataDictionary

# dataDictionary = GetVariableData(t=0)